In [ ]:
# Start writing code here...from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
spotify_songs = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')


Q1 = np.percentile(spotify_songs.duration_ms, 25,
                   interpolation = 'midpoint')
Q3 = np.percentile(spotify_songs.duration_ms, 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1

"""
mask = (spotify_songs.duration_ms < (Q3+1.5*IQR))
mask2 = spotify_songs.duration_ms > (Q1-1.5*IQR)
spotify_data_cleaned = spotify_songs.loc[mask]
spotify_data_cleaned = spotify_data_cleaned.loc[mask2]
"""
spotify_data_cleaned = spotify_songs
spotify_data_cleaned.track_album_release_date.str[:4]
spotify_data_cleaned.track_album_release_date.str[5:7]
spotify_data_cleaned = spotify_data_cleaned.assign(
    track_album_release_month = spotify_data_cleaned.track_album_release_date.str[5:7],
    track_album_release_year = spotify_data_cleaned.track_album_release_date.str[:4],
)
spotify_data_cleaned.drop(columns="track_album_release_date",inplace=True)
spotify_data_cleaned.track_album_release_year = spotify_data_cleaned.track_album_release_year.astype("int64")
"""
mask = (spotify_data_cleaned.track_popularity > 0)
spotify_data_cleaned = spotify_data_cleaned.loc[mask]
"""


'\nmask = (spotify_data_cleaned.track_popularity > 0)\nspotify_data_cleaned = spotify_data_cleaned.loc[mask]\n'

In [ ]:
spotify_data_cleaned.track_id.value_counts()

7BKLCZ1jbUBVqRi2FVlTVw    10
14sOS5L36385FJ3OL8hew4     9
3eekarcy7kvN4yt5ZFzltW     9
0sf12qNH5qcw8qpgymFOqD     8
7h0d2h0fUmzbs7zeFigJPn     8
                          ..
6ADPydLM8fmi6fjO9ZXAb8     1
0qDBPHQA0z1quCSeqQrbzn     1
3Ij3K23D4G2bceNtgxlmLn     1
2bBRrsQMO0yVhVX0mWG7G9     1
27PmvZoffODNFW2p7ehZTQ     1
Name: track_id, Length: 28356, dtype: int64

In [ ]:
indipop = spotify_data_cleaned[spotify_data_cleaned.track_id == "0sf12qNH5qcw8qpgymFOqD"]
indipop.playlist_genre.value_counts()
indipop

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,track_album_release_month,track_album_release_year
716,0sf12qNH5qcw8qpgymFOqD,Blinding Lights,The Weeknd,98,2ZfHkwHuoAZrlz7RMj0PDz,Blinding Lights,Todo Éxitos,2ji5tRQVfnhaX1w9FhmSzk,pop,dance pop,...,1,0.0629,0.00147,0.000209,0.0938,0.345,171.017,201573,11,2019
1605,0sf12qNH5qcw8qpgymFOqD,Blinding Lights,The Weeknd,98,2ZfHkwHuoAZrlz7RMj0PDz,Blinding Lights,"post-teen alternative, indie, pop (large variety)",1y42gwI5cuwjBslPyQNfqb,pop,post-teen pop,...,1,0.0629,0.00147,0.000209,0.0938,0.345,171.017,201573,11,2019
4713,0sf12qNH5qcw8qpgymFOqD,Blinding Lights,The Weeknd,98,2ZfHkwHuoAZrlz7RMj0PDz,Blinding Lights,indie poptimism🕺🏻,6MQeEHHDs6HQkN5Dlj3oKo,pop,indie poptimism,...,1,0.0629,0.00147,0.000209,0.0938,0.345,171.017,201573,11,2019
18307,0sf12qNH5qcw8qpgymFOqD,Blinding Lights,The Weeknd,98,2ZfHkwHuoAZrlz7RMj0PDz,Blinding Lights,2020 Hits & 2019 Hits – Top Global Tracks 🔥🔥🔥,4JkkvMpVl4lSioqQjeAL0q,latin,latin pop,...,1,0.0629,0.00147,0.000209,0.0938,0.345,171.017,201573,11,2019
19715,0sf12qNH5qcw8qpgymFOqD,Blinding Lights,The Weeknd,98,2ZfHkwHuoAZrlz7RMj0PDz,Blinding Lights,2020 Hits & 2019 Hits – Top Global Tracks 🔥🔥🔥,4JkkvMpVl4lSioqQjeAL0q,latin,latin hip hop,...,1,0.0629,0.00147,0.000209,0.0938,0.345,171.017,201573,11,2019
21571,0sf12qNH5qcw8qpgymFOqD,Blinding Lights,The Weeknd,98,2ZfHkwHuoAZrlz7RMj0PDz,Blinding Lights,Most Popular 2020 TOP 50,1fqkbjEACMlekdddm5aobE,r&b,urban contemporary,...,1,0.0629,0.00147,0.000209,0.0938,0.345,171.017,201573,11,2019
22933,0sf12qNH5qcw8qpgymFOqD,Blinding Lights,The Weeknd,98,2ZfHkwHuoAZrlz7RMj0PDz,Blinding Lights,Pop Hits 2020,5TDtuKDbOhrfW7C58XnriZ,r&b,hip pop,...,1,0.0629,0.00147,0.000209,0.0938,0.345,171.017,201573,11,2019
30203,0sf12qNH5qcw8qpgymFOqD,Blinding Lights,The Weeknd,98,2ZfHkwHuoAZrlz7RMj0PDz,Blinding Lights,Charts 2020 🔥Top 2020🔥Hits 2020🔥Summer 2020🔥Po...,3xMQTDLOIGvj3lWH5e5x6F,edm,pop edm,...,1,0.0629,0.00147,0.000209,0.0938,0.345,171.017,201573,11,2019


In [ ]:
index = spotify_data_cleaned.playlist_name.value_counts().reset_index()["index"]
playlist_name = []

for name in index:
    playlist_name.append(name)
playlist_name

['Indie Poptimism',
 '2020 Hits & 2019  Hits – Top Global Tracks 🔥🔥🔥',
 'Permanent Wave',
 'Hard Rock Workout',
 'Ultimate Indie Presents... Best Indie Tracks of the 2010s',
 'Fitness Workout Electro | House | Dance | Progressive House',
 'Southern Hip Hop',
 'Charts 2020 🔥Top 2020🔥Hits 2020🔥Summer 2020🔥Pop 2020🔥Popular Music🔥Clean Pop 2020🔥Sing Alongs',
 'Classic Rock 70s 80s 90s, Rock Classics - 70s Rock, 80s Rock, 90s Rock Rock  Classicos',
 'Urban Contemporary',
 'post teen pop',
 'permanent wave',
 'Dance Pop',
 'New Jack Swing',
 'Urban contemporary',
 'Tropical Vibes',
 'EDM 2020 House & Dance',
 "Gangsta Rap/90's Hip-Hop",
 'Latin Pop Classics',
 'Electropop 2019',
 'Tropical House',
 'Electro Swing Top 100',
 'Urban/Trap - Contemporary RAP',
 'Sexy Soul 2020',
 'Big Room EDM',
 'Hands Up\u200f\u200f\u200b\u200b   \u200d',
 'Chillout & Remixes 💜',
 'Reggaeton Classics',
 'Vibra Tropical',
 'Gospel Neo Soul',
 'Japanese Funk/Soul/NEO/Jazz/Acid',
 'Neo-Soul',
 'Smooth Hip Hop',
 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c2ad4461-7f35-42f5-b2d9-f686e8952f67' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>